In [23]:
import warnings

import numpy as np

from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18 as cosmo
from astropy.cosmology import z_at_value
from astropy.io import fits
from astropy.table import Table, vstack
import astropy.units as u
from astropy.utils.metadata import MergeConflictWarning
from astroquery.xmatch import XMatch
import healpy as hp
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import find_greedy_credible_levels

import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
from lsst.geom import degrees, SpherePoint

In [83]:
# need to revise to the latest collection
repo = "/repo/embargo"
collection = "u/elhoward/TOO/S251112cm/d_2025_11_15/run_02" # replace with appropriate collection
# collection = "u/elhoward/TOO/S251112cm/20251116/Rubin-Rubin"
butler = Butler(repo, collections=collection)

In [84]:
# dataref_iterator = butler.registry.queryDatasets(datasetType="difference_image", collections=[collection])

In [90]:
# refs = butler.query_datasets(
#     "dia_object", # or dia_object
#     collections="u/elhoward/TOO/S251112cm/20251116/Rubin-Rubin",
#     where="instrument='LSSTCam' AND skymap='lsst_cells_v1'", # Only include if you want to narrow things down
# )

In [99]:
# dia_obj = butler.get("dia_source", dataId=refs[10].dataId)


In [100]:
dia_obj.columns

Index(['visit', 'detector', 'band', 'diaObjectId', 'ssObjectId',
       'parentDiaSourceId', 'midpointMjdTai', 'bboxSize',
       'timeProcessedMjdTai', 'ra', 'dec', 'raErr', 'decErr', 'ra_dec_Cov',
       'x', 'y', 'xErr', 'yErr', 'apFlux', 'apFluxErr', 'snr', 'psfFlux',
       'psfFluxErr', 'psfChi2', 'psfNdata', 'trailFlux', 'trailFluxErr',
       'trailRa', 'trailDec', 'trailLength', 'trailAngle', 'dipoleMeanFlux',
       'dipoleMeanFluxErr', 'dipoleFluxDiff', 'dipoleFluxDiffErr',
       'dipoleLength', 'dipoleAngle', 'dipoleChi2', 'isDipole', 'isNegative',
       'dipoleFitAttempted', 'dipoleNdata', 'scienceFlux', 'scienceFluxErr',
       'templateFlux', 'templateFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF',
       'iyyPSF', 'ixyPSF', 'extendedness', 'reliability', 'glint_trail',
       'pixelFlags', 'pixelFlags_offimage', 'pixelFlags_edge',
       'pixelFlags_interpolated', 'pixelFlags_saturated', 'pixelFlags_cr',
       'pixelFlags_bad', 'pixelFlags_suspect', 'pixelFlags_interpolated

In [85]:
data_refs = [dr for dr in dataref_iterator.expanded()]

In [5]:
"""
good = ~dia_src["slot_Shape_flag"] & \
    (dia_src["base_PsfFlux_instFlux"] / dia_src["base_PsfFlux_instFluxErr"] > snr_threshold) & \
    ~dia_src["base_PixelFlags_flag_edge"] & \
    ((dia_src_table["scienceFlux"] / dia_src_table["scienceFluxErr"]) < max_science_snr) & \
    ~dia_src_table["pixelFlags_streak"]
"""

def good_src(cat, snr_threshold=5, max_science_snr=200):
    """Good for the dia_source_unfiltered"""
    good = ~cat["slot_Shape_flag"] & \
        (cat["base_PsfFlux_instFlux"] / cat["base_PsfFlux_instFluxErr"] > snr_threshold) & \
        ~cat["base_PixelFlags_flag_edge"]

    good_cat = cat[good].copy()

    return good_cat


## Inside-outside calculation

In [38]:
def in_skymap(skymap_prob, dr=None, good_dia_src=None, dataset_type="dia_source_unstandardized", probability = 0.95):
    if not good_dia_src:
        dia_src = butler.get(dataset_type, dataId=dr.dataId).asAstropy()
        good_dia_src = good_src(dia_src)
    ra_obj = good_dia_src["coord_ra"].to(u.deg).value
    dec_obj = good_dia_src["coord_dec"].to(u.deg).value
    npix = len(skymap_prob)
    nside = hp.npix2nside(npix)
    ipix = hp.ang2pix(nside, ra_obj, dec_obj, lonlat=True)
    credible_levels = find_greedy_credible_levels(skymap_prob)
    return credible_levels[ipix] <= probability

In [29]:
skymap = read_sky_map("../data/S251112cm_bilby.fits")
prob = skymap[0]

In [39]:
dia_srcs_inside = []
for dr in data_refs:
    dia_src = butler.get("dia_source_unstandardized", dataId=dr.dataId).asAstropy()
    good_dia_src = good_src(dia_src)
    inside = in_skymap(prob, good_dia_src=good_dia_src)
    dia_src_inside = good_dia_src[inside]
    dia_srcs_inside.append(dia_src_inside)
    

In [44]:
data_refs_inside = [dr for dr, dsm in zip(data_refs, dia_srcs_inside) if len(dsm) > 0]
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=MergeConflictWarning)
    dia_src_inside = vstack(dia_srcs_inside)

In [78]:
dia_src_inside

id,coord_ra,coord_dec,parent,is_negative,detect_isPrimary,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_PsfFlux_flag_badCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_PsfFlux_flag_badCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_notAtMaximum,base_SdssCentroid_flag_near_edge,base_CircularApertureFlux_flag_badCentroid_near_edge,base_PsfFlux_flag_badCentroid_near_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_near_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_near_edge,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_resetToPeak,base_SdssCentroid_flag_badError,base_CircularApertureFlux_flag_badCentroid_badError,base_PsfFlux_flag_badCentroid_badError,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_badError,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_badError,ip_diffim_DipoleFit_pos_instFlux,slot_Centroid_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,slot_Centroid_pos_instFluxErr,ip_diffim_DipoleFit_neg_instFlux,slot_Centroid_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,slot_Centroid_neg_instFluxErr,ip_diffim_DipoleFit_instFlux,slot_Centroid_instFlux,ip_diffim_DipoleFit_instFluxErr,slot_Centroid_instFluxErr,ip_diffim_DipoleFit_pos_x,slot_Centroid_pos_x,ip_diffim_DipoleFit_pos_y,slot_Centroid_pos_y,ip_diffim_DipoleFit_pos_xErr,slot_Centroid_pos_xErr,ip_diffim_DipoleFit_pos_yErr,slot_Centroid_pos_yErr,ip_diffim_DipoleFit_neg_x,slot_Centroid_neg_x,ip_diffim_DipoleFit_neg_y,slot_Centroid_neg_y,ip_diffim_DipoleFit_neg_xErr,slot_Centroid_neg_xErr,ip_diffim_DipoleFit_neg_yErr,slot_Centroid_neg_yErr,ip_diffim_DipoleFit_x,slot_Centroid_x,ip_diffim_DipoleFit_y,slot_Centroid_y,ip_diffim_DipoleFit_xErr,slot_Centroid_xErr,ip_diffim_DipoleFit_yErr,slot_Centroid_yErr,ip_diffim_DipoleFit_orientation,slot_Centroid_orientation,ip_diffim_DipoleFit_separation,slot_Centroid_separation,ip_diffim_DipoleFit_chi2dof,slot_Centroid_chi2dof,ip_diffim_DipoleFit_nData,slot_Centroid_nData,ip_diffim_DipoleFit_signalToNoise,slot_Centroid_signalToNoise,ip_diffim_DipoleFit_classification,slot_Centroid_classification,ip_diffim_DipoleFit_classificationAttempted,slot_Centroid_classificationAttempted,ip_diffim_DipoleFit_flag,slot_Centroid_flag,ip_diffim_DipoleFit_flag_edge,slot_Centroid_flag_edge,coord_raErr,coord_decErr,coord_ra_dec_Cov,ext_shapeHSM_HsmPsfMoments_flag,slot_PsfShape_flag,ext_shapeHSM_HsmPsfMoments_flag_no_pixels,slot_PsfShape_flag_no_pixels,ext_shapeHSM_HsmPsfMoments_flag_not_contained,slot_PsfShape_flag_not_contained,ext_shapeHSM_HsmPsfMoments_flag_parent_source,slot_PsfShape_flag_parent_source,ext_shapeHSM_HsmPsfMoments_flag_galsim,slot_PsfShape_flag_galsim,ext_shapeHSM_HsmPsfMoments_flag_invalid_param,slot_PsfShape_flag_invalid_p

In [ ]:
dia_srcs_inside.write("dia_src_inside95_S251112cm.ecsv", overwrite=True)

## NED-GWF galaxy crossmatch

In [46]:
# Load NED-GWF table
url = "https://ned.ipac.caltech.edu/uri/NED::GWFglist/fits/S251112cm/4"

ned_gwf = Table(fits.getdata(url))

In [47]:
ned_gwf

objname,ra,dec,objtype,z,z_unc,z_tech,z_qual,z_qual_flag,z_refcode,ziDist,ziDist_unc,ziDist_method,ziDist_indicator,ziDist_refcode,DistMpc,DistMpc_unc,DistMpc_method,ebv,A_FUV_MWext,A_NUV_MWext,A_J_MWext,A_H_MWext,A_Ks_MWext,A_W1_MWext,A_W2_MWext,A_W3_MWext,A_W4_MWext,m_FUV,m_FUV_unc,m_NUV,m_NUV_unc,Lum_FUV,Lum_FUV_unc,Lum_NUV,Lum_NUV_unc,GALEXphot,m_J,m_J_unc,m_H,m_H_unc,m_Ks,m_Ks_unc,Lum_J,Lum_J_unc,Lum_H,Lum_H_unc,Lum_Ks,Lum_Ks_unc,tMASSphot,m_W1,m_W1_unc,m_W2,m_W2_unc,m_W3,m_W3_unc,m_W4,m_W4_unc,Lum_W1,Lum_W1_unc,Lum_W2,Lum_W2_unc,Lum_W3,Lum_W3_unc,Lum_W4,Lum_W4_unc,WISEphot,SFR_W4,SFR_W4_unc,SFR_hybrid,SFR_hybrid_unc,ET_flag,Mstar,Mstar_unc,MLratio,P_2D,P_3D,P_2D_csum,P_3D_csum,P_Mstar,P_SFR,P_sSFR,P_LumW1,P_3D_Mstar,P_3D_SFR,P_3D_sSFR,P_3D_LumW1
str30,float64,float64,str6,float32,float32,str4,bool,bool,str19,float64,float64,str30,str1,str19,float32,float32,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,bool,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
AGC 208747,159.2775,20.5161111111111,G,0.00032689283,nan,UNKN,False,False,2016A&A...596A.117A,nan,nan,,,,1.4084027,nan,Redshift,0.019383822,0.1552131,0.17419715,0.015746957,0.009809612,0.006728239,0.003847457,0.0026839837,0.0010237546,0.0005326131,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,False,nan,nan,0.5,1.105000506557702,1.9688844829228506e-06,0.7095791258224945,0.5917971263975689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WISEA J013915.89-471751.8,24.8162476,-47.2977329,G,0.00054,0.000213,SPEC,False,False,1997AJ....114.1313C,nan,nan,,,,2.3269491,0.91746837,Redshift,0.012746077,0.102062345,0.11454555,0.010354611,0.0064504347,0.0044242386,0.002529944,0.0017648875,0.0006731828,0.00035022647,18.5441,0.0597547,18.2229,0.0352552,1.9534029466684373e+39,1.0751128615171526e+38,1.7762852484337926e+39,5.768010621731761e+37,ASC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,13.812,0.023,13.785,0.073,11.963,0.369,9.063,99.0,5.368503599881293e+38,1.1372900202386011e+37,2.2232108368809907e+38,1.4948363570663178e+37,8.731533184456437e+37,2.9676112951414574e+37,1.7435972343054267e+38,nan,APER,3.4063504186743166e-05,nan,0.00011631947236994556,nan,False,1698893.5442662318,35990.19051387978,0.5,2.294141496059659,1.0706827784233411e-06,0.5523232371090585,0.7362317337576566,9.644972019085916e-09,8.457623026546877e-09,5.7096645434090956e-05,9.524965514323448e-09,1.032670543921029e-14,9.055431320920437e-15,6.113239497202487e-11,1.0198216541262338e-14
GALEXASC J235059.08-264835.2,357.74679166667,-26.810416666669997,G,0.0006,0.000297,SPEC,True,True,20032dF...C...0000C,nan,nan,,,,2.5856192,1.2792869,Redshift,0.018108554,0.14500159,0.16273667,0.01471096,0.009164235,0.006285586,0.0035943314,0.0025074035,0.00095640146,0.0004975723,21.5012,0.345958,21.6129,0.320332,1.6469606468366552e+38,5.248035263440326e+37,1.0100544128882755e+38,2.980129881071741e+37,ASC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,False,nan,nan,0.5,3.19992875866416,2.0271092063679064e-06,0.44733279462273773,0.5854630746238607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UGC 07577,186.92042,43.495556,G,0.00065,1e-06,SPEC,False,False,2003A&A...401..483H,2.6443064212799072,0.031269438398829535,Wavg,P,Mix,2.6443064,0.03126944,zIndependent,0.017462263,0.1398265,0.15692863,0.014185928,0.008837165,0.0060612545,0.0034660504,0.0024179148,0.0009222676,0.000479814,15.018273,0.0008275661,14.63243,0.002857707,6.7185866035

In [48]:
def load_and_match_dia_src_against_cat(cat, dr=None, good_dia_src=None, dataset_type="dia_source", radius=30 * u.arcsec, debug=False):
    if not good_dia_src:
        dia_src = butler.get(dataset_type, dataId=dr.dataId).asAstropy()
        good_dia_src = good_src(dia_src)
    
    dia_coord = SkyCoord(good_dia_src["coord_ra"], good_dia_src["coord_dec"])
    cat_coord = SkyCoord(cat["ra"] * u.deg, cat["dec"] * u.deg)
    
    idx, sep2d, _ = dia_coord.match_to_catalog_sky(cat_coord)

    close_enough = sep2d < radius
    matching_idx = idx[close_enough]

    if debug:
        print(len(dia_coord))
        print(cat_coord[364])
        print(idx)
        print(sep2d)
        print(matching_idx)
        print("----")

    matched_dia_src = good_dia_src[close_enough].copy()
    matched_cat = cat[matching_idx].copy()

    return matched_dia_src, matched_cat
    

In [49]:
# Use an approximate matching radius given redshift.
z = z_at_value(cosmo.luminosity_distance, 93 * u.Mpc)
matching_radius_physical = 50 * u.kpc
matching_radius_angular = matching_radius_physical * cosmo.arcsec_per_kpc_proper(z)
print(matching_radius_physical, matching_radius_angular)

50.0 kpc 115.52553420764819 arcsec


In [50]:
# So let's call that 116"
ned_matching_radius = 116 * u.arcsec

In [58]:
ned_dia_src_matches = []
ned_cat_matches = []

for dr in data_refs_inside:
    matched_dia_src, matched_cat = load_and_match_dia_src_against_cat(ned_gwf, dr=dr, dataset_type="dia_source_unstandardized", radius=ned_matching_radius)
    ned_dia_src_matches.append(matched_dia_src)
    ned_cat_matches.append(matched_cat)

In [59]:
ned_dr_match = [dr for dr, dsm in zip(data_refs_inside, ned_dia_src_matches) if len(dsm) > 0]
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=MergeConflictWarning)
    ned_dia_src_match = vstack(ned_dia_src_matches)
    ned_cat_match = vstack(ned_cat_matches)

In [61]:
ned_dia_src_match

id,coord_ra,coord_dec,parent,is_negative,detect_isPrimary,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_PsfFlux_flag_badCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_PsfFlux_flag_badCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_notAtMaximum,base_SdssCentroid_flag_near_edge,base_CircularApertureFlux_flag_badCentroid_near_edge,base_PsfFlux_flag_badCentroid_near_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_near_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_near_edge,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_resetToPeak,base_SdssCentroid_flag_badError,base_CircularApertureFlux_flag_badCentroid_badError,base_PsfFlux_flag_badCentroid_badError,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_badError,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_badError,ip_diffim_DipoleFit_pos_instFlux,slot_Centroid_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,slot_Centroid_pos_instFluxErr,ip_diffim_DipoleFit_neg_instFlux,slot_Centroid_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,slot_Centroid_neg_instFluxErr,ip_diffim_DipoleFit_instFlux,slot_Centroid_instFlux,ip_diffim_DipoleFit_instFluxErr,slot_Centroid_instFluxErr,ip_diffim_DipoleFit_pos_x,slot_Centroid_pos_x,ip_diffim_DipoleFit_pos_y,slot_Centroid_pos_y,ip_diffim_DipoleFit_pos_xErr,slot_Centroid_pos_xErr,ip_diffim_DipoleFit_pos_yErr,slot_Centroid_pos_yErr,ip_diffim_DipoleFit_neg_x,slot_Centroid_neg_x,ip_diffim_DipoleFit_neg_y,slot_Centroid_neg_y,ip_diffim_DipoleFit_neg_xErr,slot_Centroid_neg_xErr,ip_diffim_DipoleFit_neg_yErr,slot_Centroid_neg_yErr,ip_diffim_DipoleFit_x,slot_Centroid_x,ip_diffim_DipoleFit_y,slot_Centroid_y,ip_diffim_DipoleFit_xErr,slot_Centroid_xErr,ip_diffim_DipoleFit_yErr,slot_Centroid_yErr,ip_diffim_DipoleFit_orientation,slot_Centroid_orientation,ip_diffim_DipoleFit_separation,slot_Centroid_separation,ip_diffim_DipoleFit_chi2dof,slot_Centroid_chi2dof,ip_diffim_DipoleFit_nData,slot_Centroid_nData,ip_diffim_DipoleFit_signalToNoise,slot_Centroid_signalToNoise,ip_diffim_DipoleFit_classification,slot_Centroid_classification,ip_diffim_DipoleFit_classificationAttempted,slot_Centroid_classificationAttempted,ip_diffim_DipoleFit_flag,slot_Centroid_flag,ip_diffim_DipoleFit_flag_edge,slot_Centroid_flag_edge,coord_raErr,coord_decErr,coord_ra_dec_Cov,ext_shapeHSM_HsmPsfMoments_flag,slot_PsfShape_flag,ext_shapeHSM_HsmPsfMoments_flag_no_pixels,slot_PsfShape_flag_no_pixels,ext_shapeHSM_HsmPsfMoments_flag_not_contained,slot_PsfShape_flag_not_contained,ext_shapeHSM_HsmPsfMoments_flag_parent_source,slot_PsfShape_flag_parent_source,ext_shapeHSM_HsmPsfMoments_flag_galsim,slot_PsfShape_flag_galsim,ext_shapeHSM_HsmPsfMoments_flag_invalid_param,slot_PsfShape_flag_invalid_p

In [ ]:
ned_dia_src_match.write("dia_src_match_116arcsec_S251112cm.ecsv", overwrite=True)

## Gaia crossmatch

In [61]:
# check what Gaia DR3 catalogs are available in astroquery.XMatch using the CDS identifier
catalogs_available = XMatch.get_available_tables()
for name in catalogs_available:
    if "I/358" in name:
        print(name)

I/358/veprv
I/358/varisum
I/358/vmsosc
II/358/smss
I/358/vrvstat
I/358/vlpv
I/358/vagn
I/358/vcep
I/358/vpltrans
I/358/vmicro
I/358/scalerts
I/358/vrm
I/358/veb
I/358/vcc
I/358/vst
I/358/alertsms
I/358/vrrlyr
I/358/vclassre


In [69]:
def load_and_match_dia_src_against_gaia(dr = None, good_dia_src = None, dataset_type="dia_source", radius= 3.0 * u.arcsec):
    """Option to get good dia sources from input data ref or input a filtered list of dia sources"""
    if not good_dia_src:
        dia_src = butler.get("dia_source_unstandardized", dataId=dr.dataId).asAstropy()
        good_dia_src = good_src(dia_src)
        
    input_table = Table([good_dia_src['id'], good_dia_src['coord_ra'].to(u.deg).value, good_dia_src['coord_dec'].to(u.deg).value], names=['id', 'ra', 'dec'])

    radius_deg = radius.to(u.deg).value

    # use vizier:I/355/gaiadr3 for excluding all Gaia crossmatches
    gaia_table = XMatch.query(cat1=input_table, cat2="vizier:I/358/scalerts", max_distance=2 * u.arcsec, colRA1='ra', colDec1='dec')

    if len(gaia_table) > 0:
        mask = good_dia_src['id'] == gaia_table['id']
        unmatched_dia_src = good_dia_src[mask^True] # reject sources that are within 2 " of GAIA DR3 variable sources
    else:
        unmatched_dia_src = good_dia_src
        
    return unmatched_dia_src, gaia_table
    

In [71]:
gaia_cat_matches = []
dia_srcs_filtered_gaia = []
for dr in ned_dr_match:
    unmatched_dia_src, matched_cat = load_and_match_dia_src_against_gaia(dr=dr, dataset_type="dia_source_unstandardized")
    dia_srcs_filtered_gaia.append(unmatched_dia_src)
    gaia_cat_matches.append(matched_cat)

In [74]:
gaia_dr_filtered = [dr for dr, dsm in zip(ned_dr_match, dia_srcs_filtered_gaia) if len(dsm) > 0]
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=MergeConflictWarning)
    dia_src_filtered_gaia = vstack(dia_srcs_filtered_gaia)
    gaia_cat_match = vstack(gaia_cat_matches)

In [75]:
dia_src_filtered_gaia

id,coord_ra,coord_dec,parent,is_negative,detect_isPrimary,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_PsfFlux_flag_badCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_PsfFlux_flag_badCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_notAtMaximum,base_SdssCentroid_flag_near_edge,base_CircularApertureFlux_flag_badCentroid_near_edge,base_PsfFlux_flag_badCentroid_near_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_near_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_near_edge,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_resetToPeak,base_SdssCentroid_flag_badError,base_CircularApertureFlux_flag_badCentroid_badError,base_PsfFlux_flag_badCentroid_badError,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_badError,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_badError,ip_diffim_DipoleFit_pos_instFlux,slot_Centroid_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,slot_Centroid_pos_instFluxErr,ip_diffim_DipoleFit_neg_instFlux,slot_Centroid_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,slot_Centroid_neg_instFluxErr,ip_diffim_DipoleFit_instFlux,slot_Centroid_instFlux,ip_diffim_DipoleFit_instFluxErr,slot_Centroid_instFluxErr,ip_diffim_DipoleFit_pos_x,slot_Centroid_pos_x,ip_diffim_DipoleFit_pos_y,slot_Centroid_pos_y,ip_diffim_DipoleFit_pos_xErr,slot_Centroid_pos_xErr,ip_diffim_DipoleFit_pos_yErr,slot_Centroid_pos_yErr,ip_diffim_DipoleFit_neg_x,slot_Centroid_neg_x,ip_diffim_DipoleFit_neg_y,slot_Centroid_neg_y,ip_diffim_DipoleFit_neg_xErr,slot_Centroid_neg_xErr,ip_diffim_DipoleFit_neg_yErr,slot_Centroid_neg_yErr,ip_diffim_DipoleFit_x,slot_Centroid_x,ip_diffim_DipoleFit_y,slot_Centroid_y,ip_diffim_DipoleFit_xErr,slot_Centroid_xErr,ip_diffim_DipoleFit_yErr,slot_Centroid_yErr,ip_diffim_DipoleFit_orientation,slot_Centroid_orientation,ip_diffim_DipoleFit_separation,slot_Centroid_separation,ip_diffim_DipoleFit_chi2dof,slot_Centroid_chi2dof,ip_diffim_DipoleFit_nData,slot_Centroid_nData,ip_diffim_DipoleFit_signalToNoise,slot_Centroid_signalToNoise,ip_diffim_DipoleFit_classification,slot_Centroid_classification,ip_diffim_DipoleFit_classificationAttempted,slot_Centroid_classificationAttempted,ip_diffim_DipoleFit_flag,slot_Centroid_flag,ip_diffim_DipoleFit_flag_edge,slot_Centroid_flag_edge,coord_raErr,coord_decErr,coord_ra_dec_Cov,ext_shapeHSM_HsmPsfMoments_flag,slot_PsfShape_flag,ext_shapeHSM_HsmPsfMoments_flag_no_pixels,slot_PsfShape_flag_no_pixels,ext_shapeHSM_HsmPsfMoments_flag_not_contained,slot_PsfShape_flag_not_contained,ext_shapeHSM_HsmPsfMoments_flag_parent_source,slot_PsfShape_flag_parent_source,ext_shapeHSM_HsmPsfMoments_flag_galsim,slot_PsfShape_flag_galsim,ext_shapeHSM_HsmPsfMoments_flag_invalid_param,slot_PsfShape_flag_invalid_p

## WISE / Milliquas crossmatch

In [14]:
# check what Gaia DR3 catalogs are available in astroquery.XMatch using the CDS identifier
for name in catalogs_available:
    if "J/ApJS/234/23/r90cat" in name:
        print(name)

## Distance consistency (LS crossmatch / distant galaxies in the refcat)

In [ ]:
# 0.8 " is a typical size for galaxy in Rubin
# 0.145 kpc / arcsec appropriate for the distance to the galaxy
# discard hosts that < 1 " across -- unlikely to be associated??